In [ ]:
import math

In [ ]:
input_network = '''-,14,10,19,-,-,-
14,-,-,15,18,-,-
10,-,-,26,,29,-
19,15,26,-,16,17,21
-,18,-,16,-,-,9
-,-,29,17,-,-,25
-,-,-,21,9,25,-
'''

# input_network = '''-,-,-
# -,-,1
# -,1,-
# '''


In [60]:



class Graph:
    def __init__(self, nodes: list):
        self.nodes: list = nodes.copy()
        self.edges_matrix: list[list] = []
        for i in self.nodes:
            self.edges_matrix.append([None for _ in self.nodes])
        
        
    def connect_node(self, i: int, j: int, weight: int):
        assert(i < len(self.nodes)), f"The node {i} must be in the network"
        assert(j < len(self.nodes)), f"The node {j} must be in the network"
        self.edges_matrix[self.nodes.index(i)][self.nodes.index(j)] = weight
        self.edges_matrix[self.nodes.index(j)][self.nodes.index(i)] = weight
    
    def add_node(self, node) -> bool:
        if node in self.nodes:
            return False
        for edge in self.edges_matrix:
            edge.append(None)
        self.nodes.append(node)
        self.edges_matrix.append([None for _ in self.nodes])
        
    def nodes_connected(self, node1, node2, currently_visited = [])-> bool:
        node1_index = self.nodes.index(node1)
        node2_index = self.nodes.index(node2)
        for i, edge in enumerate(self.edges_matrix[node1_index]):
            if edge != None:
                if i == node2_index:
                    return True
                if self.nodes[i] in currently_visited:
                    continue
                currently_visited.append(node1)
                return self.nodes_connected(self.nodes[i], node2, currently_visited)
        return False
    
    def print(self):
        print("Nodes", self.nodes)
        print("Edges: [")
        for edge in self.edges_matrix:
            print(edge)
        print("]: Edges")
        
    def get_connected_subgraph(self) -> list["Graph"]:
        list_connected_subgraphs: list[Graph] = []
        for node in self.nodes:
            if (Graph.in_one_of_subgraphs(node, list_connected_subgraphs)) != None:
                continue
            for subgraph in list_connected_subgraphs:
                if(self.nodes_connected(node, subgraph.nodes[0], [])):
                    subgraph.add_node(node)
                    break
            else:
            
                list_connected_subgraphs.append(Graph([node]))
        # for i, subgraph in enumerate(list_connected_subgraphs):
        #     for node1 in subgraph.nodes:
        #         for node2 in subgraph.nodes:
        #             node1_index = self.nodes.index(node1)
        #             node2_index = self.nodes.index(node2)
        #             print(node1, node1_index, node2, node2_index)
        #             edge_weight = self.edges_matrix[node1_index][node2_index]
        #             if edge_weight != None:
        #                 subgraph.connect_node(node1, node2, edge_weight)
        return list_connected_subgraphs
            
            
        
        
    def in_one_of_subgraphs(node, sub_graphs: list["Graph"]) -> int|None:
        for i, sub_graph in enumerate(sub_graphs):
            if node in sub_graph.nodes:
                return i
            
        return None
        
        
def parse_network(input_str: str) -> Graph:
    lines = input_str.strip().split('\n')
    edges = []
    for line in lines:
        row = []
        for item in line.split(','):
            if item == '-' or item == '':
                row.append(None)
            else:
                row.append(int(item))
        edges.append(row)
    
    assert(len(edges) == len(edges[0])), "Network must be square"
    nodes = [i for i in range(len(edges))]
    graph = Graph(nodes)
    graph.edges_matrix = edges
    return graph

def min_edge(edges: list[int])-> tuple[int, int]:
    min_value = math.inf
    node_index = -1
    for i, edge in enumerate(edges):
        if edge != None and edge < min_value:
            min_value = edge
            node_index = i
    return node_index, min_value
        
print()

In [55]:
input_graph = parse_network(input_network)
no_of_nodes = len(input_graph.nodes)
nodes_edges = {}
input_graph.print()


Nodes [0, 1, 2, 3, 4, 5, 6]
Edges: [
[None, 14, 10, 19, None, None, None]
[14, None, None, 15, 18, None, None]
[10, None, None, 26, None, 29, None]
[19, 15, 26, None, 16, 17, 21]
[None, 18, None, 16, None, None, 9]
[None, None, 29, 17, None, None, 25]
[None, None, None, 21, 9, 25, None]
]: Edges


In [56]:
for i in input_graph.nodes:
    nodes_edges[i] = min_edge(input_graph.edges_matrix[i])
    
print(nodes_edges)

{0: (2, 10), 1: (0, 14), 2: (0, 10), 3: (1, 15), 4: (6, 9), 5: (3, 17), 6: (4, 9)}


In [57]:
out_graph = Graph( input_graph.nodes)
out_graph.nodes =  input_graph.nodes.copy()

for node, edge_node in nodes_edges.items():
    if edge_node[0] == -1:
        continue
    out_graph.connect_node(node, edge_node[0], edge_node[1])
    print(node, edge_node)
out_graph.print()

0 (2, 10)
1 (0, 14)
2 (0, 10)
3 (1, 15)
4 (6, 9)
5 (3, 17)
6 (4, 9)
Nodes [0, 1, 2, 3, 4, 5, 6]
Edges: [
[None, 14, 10, None, None, None, None]
[14, None, None, 15, None, None, None]
[10, None, None, None, None, None, None]
[None, 15, None, None, None, 17, None]
[None, None, None, None, None, None, 9]
[None, None, None, 17, None, None, None]
[None, None, None, None, 9, None, None]
]: Edges


In [71]:
for sub in (out_graph.get_connected_subgraph()):
    sub.print()
    pass

Nodes [0, 1, 2, 3]
Edges: [
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
]: Edges
Nodes [4, 6]
Edges: [
[None, None]
[None, None]
]: Edges
Nodes [5]
Edges: [
[None]
]: Edges


In [59]:
max_saving = maximum_saving(input_network)
print(max_saving)

NameError: name 'maximum_saving' is not defined